In [1]:
#import findspark
#findspark.init('/usr/local/spark')
# FINDSPARK BREAKS THE SETTINGS FOR RUNNING ON THE MASTER+WORKERS
# INSTEAD, DO:
import os, sys
sys.path[:0] = ['/usr/local/spark/python', '/usr/local/spark/python/lib/py4j-0.10.9-src.zip']

from pyspark import SparkConf, SparkContext

In [2]:
conf = SparkConf()
# comment next line to use Spark locally
#conf.setMaster("spark://spark-master:7077")
#conf.set("spark.executor.memory", "1g")
sc = SparkContext(conf=conf)


In [3]:
def indirect_friends(tuple):
    user, friends = tuple
    for friend1 in friends:
        for friend2 in friends:
            if friend2 != friend1:
                yield ((friend1, friend2), 1)

In [6]:
# Read file from HDFS (hdfs://namenode:8020/user/root/soc-LiveJournal1Adj.txt.gz)
#text_file = sc.textFile("soc-LiveJournal1Adj.txt.gz")

# Read file from local FS (use Spark locally, not the cluster)
text_file = sc.textFile("../data/soc-LiveJournal1Adj.txt")

In [7]:
data = text_file.filter(lambda e: "\t" in e).map(lambda l: l.split("\t")).map(lambda l: (int(l[0]), [int(f) for f in l[1].strip().split(",") if l[1].strip() != '']))
data = data.sample(withReplacement=False, fraction=0.01)
# cache data to use in two transformations
data.cache()

PythonRDD[6] at RDD at PythonRDD.scala:53

In [8]:
friends = data.flatMapValues(lambda friend: friend).map(lambda p: ((p[0], p[1]), 1))

In [7]:
suggestions = data.flatMap(indirect_friends).reduceByKey(lambda v1, v2: v1 + v2)

In [8]:
suggestions = suggestions.subtractByKey(friends).sortBy(lambda x: x[1], False)

In [9]:
suggestions.take(10)

[((16539, 40423), 4),
 ((40423, 16539), 4),
 ((36908, 36936), 3),
 ((36936, 36908), 3),
 ((13867, 14231), 3),
 ((13867, 14237), 3),
 ((13959, 14071), 3),
 ((14071, 13959), 3),
 ((14231, 13867), 3),
 ((14231, 14237), 3)]